In [2]:
# once i have wholegenome_offset.trees
import pandas as pd
import tskit
import allel
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tsinfer
import pyslim
import json
import os

In [3]:
def overlap_neutral_mut (ts_new, ts, mapper_realid_metadataid):
    ## extract surviving ndoes and comapre them to our old ndoes to place mtuations in the right place
    surviving_nodes = []
    for i in ts_new.tables.nodes:
        surviving_nodes.append(i.metadata['slim_id'])
    ## new nodes id and the ids i gave them in the past
    new_mapper = pd.DataFrame({'new_ids': range(0, len(ts_new.tables.nodes)), 'my_ids_metadata':surviving_nodes})
    ## map old nodes with new nodes
    mapper_lost_nodes = new_mapper.merge(mapper_realid_metadataid, left_on = 'my_ids_metadata', right_on = 'my_ids_metadata', how= 'right')

    ## create a mask to only keep from the old nodes the ones that survived the simulation
    mask = mapper_lost_nodes['new_ids'].notna()

    tables_og = ts.dump_tables()

    ## now filter old tables only based on surviving nodes 
    tables_og.nodes.replace_with(tables_og.nodes[mask])

    ## now filter mutation table based on the surviving nodes, for that, extract the nodes 
    old_nodes = tables_og.mutations.node

    old_nodes = pd.Series(old_nodes)

    old_nodes.name = 'old_nodes'

    ## create a dataframe relating the new and old nodes
    replace_oldbynew_nodes = pd.merge(old_nodes, mapper_lost_nodes, left_on ='old_nodes', right_on = 'real_id', how= 'left')

    ## create a mask to filter out all the mutations than has been lost 
    mask_mutations_lost = replace_oldbynew_nodes['new_ids'].notna()

    ## filter out mutations that has been lost 
    table_mutations = tables_og.mutations[mask_mutations_lost]

    ## now replace the old nodes ids by the new nodes ids with the mapper
    ids_to_replace = replace_oldbynew_nodes.dropna()['new_ids']
    table_mutations.node = np.array(ids_to_replace.astype('int32'))

    ## and jsut set the sites from 0 to the length of mutation table 
    table_mutations.site = np.array(range(0, len(table_mutations))).astype('int32')

    ## apply the same filter from the mutations table to the sites table 
    table_sites = tables_og.sites[mask_mutations_lost]  

    ## now replace all this filter old tables in the new tree seq! 
    new_tables = ts_new.dump_tables()

    new_tables.mutations.replace_with(table_mutations)

    new_tables.sites.replace_with(table_sites)

    ## make sure to compure mutations parents
    new_tables.compute_mutation_parents()

    ## create tree seq based on tables
    tree_nm = new_tables.tree_sequence()

    return tree_nm.simplify()

In [4]:
def convert_tree_to_vcf (tree,name_vcf):
    # create a vcf file from the treeseq 
    with open(name_vcf, 'w') as file:
        # Pass the file object as the output parameter
        tree.write_vcf(output=file)

In [32]:
#import the old tree
ts_old = tskit.load("../../treeseq/two_samples_baselinetree.trees")
#import mapper old nodes to new nodes
mapper_realid_metadataid = pd.read_csv('../../treeseq/mapper_realid_metadataid_two_samples.csv')

In [33]:
cloning = tskit.load('og_tree_after_cloning.trees')

In [34]:
ts_nm = overlap_neutral_mut(cloning, ts_old, mapper_realid_metadataid)

In [35]:
convert_tree_to_vcf(ts_nm, f'cloning.vcf')

In [36]:
selfing = tskit.load('og_tree_after_selfing.trees')

ts_nm = overlap_neutral_mut(selfing, ts_old, mapper_realid_metadataid)

convert_tree_to_vcf(ts_nm, f'selfing.vcf')

In [37]:
vcf_cloning = allel.read_vcf('cloning.vcf')
## extract the genotype from the vcf file
geno_cloning = vcf_cloning["calldata/GT"]
## calcualte the genotype counts

In [38]:
len(vcf_cloning['samples'])

4

In [39]:
geno_cloning = np.swapaxes(geno_cloning, 0, 1)
## count ecotypes 
genotpye_counts_c = {}
for i in geno_cloning:
    sample = i.tobytes()
    if sample in genotpye_counts_c:
        genotpye_counts_c[sample] += 1
    else:
        genotpye_counts_c[sample] = 1

In [40]:
genotpye_counts_c.values()

dict_values([2, 2])

In [41]:
len(genotpye_counts_c)

2

In [42]:
vcf_selfing = allel.read_vcf('selfing.vcf')
## extract the genotype from the vcf file
geno_selfing = vcf_selfing["calldata/GT"]
## calcualte the genotype counts

In [43]:
len(vcf_selfing['samples'])

4

In [44]:
geno_selfing = np.swapaxes(geno_selfing, 0, 1)
## count ecotypes 
genotpye_counts_s = {}
for i in geno_selfing:
    sample = i.tobytes()
    if sample in genotpye_counts_s:
        genotpye_counts_s[sample] += 1
    else:
        genotpye_counts_s[sample] = 1

In [45]:
genotpye_counts_s.values()

dict_values([2, 2])

In [47]:
genoc_selfing.value_counts()

1    368
2     47
dtype: int64

In [20]:
len(genotpye_counts_s)

415

In [22]:
415 - 231

184

In [ ]:
pd.read_csv()